# Kredi Kartı Dolandırıcılığı Tespiti

Önceki egzersizler, bir sinir ağının tüm farklı bileşenlerine daha yakından bakmanızı sağladı:
* sıralı (sequential) yoğun (Dense) bir Sinir Ağı mimarisi,
* derleme (compilation) yöntemi,
* modelin eğitilmesi (fitting).

Şimdi **çok fazla veri içeren** gerçek hayat bir veri seti üzerinde çalışalım!

**Veri seti: `Kredi Kartı İşlemleri (Credit Card Transactions)`**

Bu açık uçlu challenge’da, **kredi kartı işlemlerinden elde edilmiş verilerle** çalışacaksınız.

Bu veriler **hassas** olduğu için, toplam 31 sütundan yalnızca 3’ü bilinmektedir; geri kalanlar verileri **anonimleştirmek** amacıyla dönüştürülmüştür (aslında bunlar, orijinal verilerin **PCA projeksiyonlarıdır**).

Bilinen 3 sütun şunlardır:

* `TIME`: İşlemin, veri setindeki ilk işleme göre geçen süresi  
* `AMOUNT`: İşlem tutarı  
* `CLASS` (hedef değişkenimiz):
    * `0 : geçerli işlem`
    * `1 : sahte (fraud) işlem`

❓ **Soru** ❓ Veri setini indirerek başlayın:
* Kaggle üzerinden: [buradan](https://www.kaggle.com/mlg-ulb/creditcardfraud)
* veya bizim URL’mizden: [buradan](https://d32aokrjazspmn.cloudfront.net/materials/creditcard.csv)

Veriyi yükleyerek `X` ve `y` değişkenlerini oluşturun.

## 1. Sınıfların yeniden dengelenmesi

In [1]:
import pandas as pd

data=pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
X=data.drop(columns=['Class'])
y=data['Class']


In [3]:
# Sınıf dengesini kontrol edelim
pd.Series(y).value_counts(normalize=True)

,proportion
Class,
0,0.998273
1,0.001727


☝️ Bu `fraud detection` (sahtekârlık tespiti) challenge’ında **sınıflar aşırı derecede dengesizdir**:
* %99.8 normal işlemler
* %0.2 sahte (fraud) işlemler

**Ciddi yeniden dengeleme (rebalancing) stratejileri uygulamazsak sahtekârlık vakalarını tespit edemeyiz!**

❓ **Soru** ❓
1. **Önce**, veri setinizden üç ayrı bölme oluşturun: `Train / Val / Test`.  
   Doğrulama (validation) ve test setlerinin **dengesiz** kalması son derece önemlidir; böylece modeli değerlendirirken gerçek koşullar korunur ve veri sızıntısı (data leakage) oluşmaz. Test setinizi bu notebook’un **en son hücresine kadar saklayın**!

&nbsp;

2. **İkinci olarak**, yalnızca eğitim setinizi (training set) yeniden dengeleyin. Birçok seçeneğiniz var:

- Azınlık sınıfını rastgele oversample etmek (düz NumPy fonksiyonlarıyla).  
  *(En iyi seçenek değildir; çünkü satırları kopyalayarak veri sızıntısı yaratır.)*
- <a href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">**`Synthetic Minority Oversampling Technique - SMOTE`**</a> kullanarak, mevcut gözlemleri ağırlıklandırıp yeni veri noktaları üretmek
- Buna ek olarak, çoğunluk sınıfını bir miktar küçültmek için  
  <a href="https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/">**`RandomUnderSampler`**</a> da deneyebilirsiniz

In [4]:
from sklearn.model_selection import train_test_split

X_train,X_temp,y_train,y_temp = train_test_split(
    X,
    y,
    stratify=y,
    test_size=0.3,
    random_state=42
)

X_val,X_test,y_val,y_test = train_test_split(
    X_temp,
    y_temp,
    stratify=y_temp,
    test_size=0.5,
    random_state=42
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"Train Dolandırıcılık Oranı: {y_train.mean():.4f}")
print(f"Test Dolandırıcılık Oranı: {y_test.mean():.4f}")

Train: (199364, 30), Val: (42721, 30), Test: (42722, 30)
Train Dolandırıcılık Oranı: 0.0017
Test Dolandırıcılık Oranı: 0.0017


In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_train_res,y_train_res=smote.fit_resample(X_train,y_train)

print("Yeniden Dengelenmiş Eğitim Seti:")
print(y_train_res.value_counts())
print(f"\nEski örnek sayısı: {len(X_train)}")
print(f"Yeni örnek sayısı: {len(X_train_res)}")
print(f"Test Dolandırıcılık Oranı: {y_train_res.mean():.4f}")

Yeniden Dengelenmiş Eğitim Seti:
Class
0    199020
1    199020
Name: count, dtype: int64

Eski örnek sayısı: 199364
Yeni örnek sayısı: 398040
Test Dolandırıcılık Oranı: 0.5000


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Ölçeklendirilmiş ilk 5 değer:\n", X_train_scaled[0][:5])

Ölçeklendirilmiş ilk 5 değer:
 [ 1.38319664  0.44982353 -0.34382958  0.62280107 -0.98470239]


## 2. Sinir Ağı yinelemeleri

Sınıflarınızı yeniden dengelediğinize göre, test puanınızı optimize etmek için bir sinir ağı uyarlayın. Aşağıdaki ipuçlarını kullanmaktan çekinmeyin:

- Girişlerinizi normalleştirin!
    - Modelinizdeki ön işlemeyi “boru hattı” haline getirmek için tercihen model içinde bir [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) katmanı kullanın.
    - Veya modelinizin dışında sklearn'in [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) öğesini kullanın, `X_train`, `X_val` ve `X_test` öğelerini uygulayın.
- Modelinizi aşırı uyumlu hale getirin, ardından aşağıdakileri kullanarak düzenleyin:
- Erken Durdurma kriterleri
- [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) katmanları
    - veya [`düzenleyiciler`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) katmanları
- 🚨 İzlemek istediğiniz metrikleri ve kullanmak istediğiniz kayıp fonksiyonunu dikkatlice düşünün!

In [16]:
from tensorflow.keras import models,layers,metrics

def initialize_model():
    model = models.Sequential()

    #giriş ve gizli katman
    model.add(layers.Dense(32,activation='relu',input_shape=(X_train_scaled.shape[1],)))
    model.add(layers.Dropout(0.2))# Rastgele nöronları kapatarak ezberlemeyi önler

    #2. gizli katman
    model.add(layers.Dense(16,activation='relu'))
    model.add(layers.Dropout(0.2))

    #çıkış katmanı(binary classifaction için sigmoid)
    model.add(layers.Dense(1,activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[metrics.Recall(),
        metrics.Precision()]) # Corrected `metrics.precision` to `metrics.Precision`
    return model

model = initialize_model()
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 32)             │           992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,537 (6.00 KB)

 Trainable params: 1,537 (6.00 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

#1.erken durdurma
es=EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=10,
    restore_best_weights=True)

history=model.fit(
    X_train_scaled,
    y_train_res,
    validation_data=(X_val_scaled,y_val),
    epochs=100,
    batch_size=64,
    callbacks=[es],
    verbose=1
)

Epoch 1/100
6220/6220 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.1125 - precision: 0.9801 - recall: 0.9324 - val_loss: 0.0255 - val_precision: 0.1902 - val_recall: 0.8378
Epoch 2/100
6220/6220 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0253 - precision: 0.9915 - recall: 0.9932 - val_loss: 0.0186 - val_precision: 0.3043 - val_recall: 0.8514
Epoch 3/100
6220/6220 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0159 - precision: 0.9941 - recall: 0.9967 - val_loss: 0.0169 - val_precision: 0.2995 - val_recall: 0.8378
Epoch 4/100
6220/6220 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.0125 - precision: 0.9952 - recall: 0.9979 - val_loss: 0.0163 - val_precision: 0.4800 - val_recall: 0.8108
Epoch 5/100
6220/6220 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.0112 - precision: 0.9960 - recall: 0.9981 - val_loss: 0.0179 - val_precision: 0.4150 - val_recall: 0.8243
Epoch 6/100
6220/6220 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 0.0098 - precision: 0.9960 - recall: 0.9984 - val_loss: 0.0172 - val_precisi

In [20]:
# Modelin test seti üzerindeki performansını ölç
results = model.evaluate(X_test_scaled, y_test, verbose=0)

# Metrik isimlerini ve sonuçları eşleştirelim
precision = results[2] # Model compile ederken metrics listesindeki sıraya göre
recall = results[1]

print(f"Final Test Keskinliği (Precision): {precision:.4f}")
print(f"Final Test Duyarlılığı (Recall): {recall:.4f}")

Final Test Keskinliği (Precision): 0.4692
Final Test Duyarlılığı (Recall): 0.8243


### 🧪 Kodunu Test Et

Orijinal dengesiz veri kümesinin (`X_test`, `y_test`) temsil edici bir örneğinde gerçek test performansınızın altında kalan sonuçları `precision` ve `recall` değişkenlerine kaydedin.

In [22]:
!pip install nbresult

In [25]:
# GitHub deposunu tamamen indir
!git clone https://github.com/lfunnyl/S18D3-S-Data-credit-card-challenge

# İndirilen klasörün içindeki tests klasörünü ana dizine kopyala
!cp -r S18D3-S-Data-credit-card-challenge/tests .

Cloning into 'S18D3-S-Data-credit-card-challenge'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 4.11 KiB | 1.03 MiB/s, done.


In [26]:
from nbresult import ChallengeResult

result = ChallengeResult('solution',
    precision=precision,
    recall=recall,
    fraud_number=len(y_test[y_test == 1]),
    non_fraud_number=len(y_test[y_test == 0]),
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/tests
plugins: anyio-4.12.1, typeguard-4.4.4, langsmith-0.6.1
collecting ... collected 2 items

test_solution.py::TestSolution::test_is_score_good_enough PASSED         [ 50%]
test_solution.py::TestSolution::test_is_test_set_representative PASSED   [100%]

============================== 2 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## 🏁 İsteğe bağlı: Bu zorluk için Google'ın çözümünü okuyun
Bu oturumdaki tüm zorlukları tamamladığınız için tebrikler!

Son olarak, Google'ın kendi çözümünü doğrudan [Colab'da buradan](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb) okuyun.

İlginç teknikler ve en iyi uygulamaları keşfedeceksiniz.